# basicprobabilistic CPU - JIT

In [3]:
import math
import random
from numba import cuda,jit,njit
import numpy as np
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import time
import matplotlib.pyplot as plt   
import sys
import os
sys.path.append("/home/servelgar/tfg-cuda/vm/examples")
sys.path.append("/home/servelgar/tfg-cuda/vm/src")
sys.path.append("/home/servelgar/tfg-cuda/vm/tests")
print(sys.path)
from basicprobabilistic import basicprobabilistic
from spsuma import spsuma


['/home/servelgar/tfg-cuda/src', '/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/servelgar/.local/lib/python3.9/site-packages', '/usr/local/lib64/python3.9/site-packages', '/usr/local/lib/python3.9/site-packages', '/usr/lib64/python3.9/site-packages', '/usr/lib/python3.9/site-packages', '/home/servelgar/tfg-cuda/vm/examples', '/home/servelgar/tfg-cuda/vm/src', '/home/servelgar/tfg-cuda/vm/tests', '/home/servelgar/tfg-cuda/vm/examples', '/home/servelgar/tfg-cuda/vm/src', '/home/servelgar/tfg-cuda/vm/tests']


In [4]:
@njit
def cpu_simulaciones(n,p,array_sims):
    
    cdf = np.array([p,(p+(1-p))])
    
    for i in range(array_sims.shape[0]):
        for j in range(n):
            array_sims[i][0] -= 1
            rnd = random.random()
            index = 0
            while rnd > cdf[index]:
                index += 1
            array_sims[i][index+1] += 1
    
    return array_sims


In [5]:
@njit
def cpu_simulaciones(n,p,array_sims):
    
    cdf = np.array([p,(p+(1-p))])
    
    for i in range(array_sims.shape[0]):
        for j in range(n):
            array_sims[i][0] -= 1
            rnd = random.random()
            index = 0
            while rnd > cdf[index]:
                index += 1
            array_sims[i][index+1] += 1
    
    return array_sims


In [10]:
def solucion_basicprobabilistic_sims(array_sims):
    tam = array_sims.shape[0]
    s = f"basicprobabilistic: {tam} simulaciones\n"
    s += f"nº virus: {n}\n"
    s += f"probabilidad de pasar a host2: {p*100} %\n"
    s += f"probabilidad de pasar a host3: {(1-p)*100} %\n"
    s += "----------------------------------------\n"
    if (tam <= 10):
        for i in range(array_sims.shape[0]):
            s += f"Simulación: {i+1}\n"
            s += "HOST 1 | HOST 2 | HOST 3\n"
            s += f"{array_sims[i][0]} | {array_sims[i][1]} | {array_sims[i][2]}\n"
            s += "++++++++++++++++++++++++++++++++++\n"
    else:
        tam = 5
        for i in range(tam):
            s += f"Simulación: {i+1}\n"
            s += "HOST 1 | HOST 2 | HOST 3\n"
            s += f"{array_sims[i][0]} | {array_sims[i][1]} | {array_sims[i][2]}\n"
            s += "++++++++++++++++++++++++++++++++++\n"
        for i in range(tam,0,-1):
            if i == tam:
                s += ".\n"
                s += ".\n"
                s += ".\n"
            s += f"Simulación: {array_sims.shape[0]-i + 1}\n"
            s += "HOST 1 | HOST 2 | HOST 3\n"
            s += f"{array_sims[array_sims.shape[0]-i-1][0]} | {array_sims[array_sims.shape[0]-i-1][1]} | {array_sims[array_sims.shape[0]-i-1][2]}\n"
            s += "++++++++++++++++++++++++++++++++++\n"
    return s

In [11]:
def solucion_basicprobabilistic_medias(array_media):
    tam = array_media.shape[0]
    s = f"basicprobabilistic: {tam} simulaciones\n"
    s += "probabilidad acumulada entre simulaciones\n"
    s += "probabilidad teórica: \n"
    s += f"host2: {p*100} % | host3: {(1-p)*100} %\n"
    s += "----------------------------------------\n"
    if(tam <= 10):
        for i in range(array_media.shape[0]):
            s += f"Simulación: {i+1}\n"
            s += "PROB. ACUM. HOST 2 | PROB. ACUM. HOST 3\n"
            s += f"  {array_media[i][0]*100} % |     {array_media[i][1]*100} %\n"
            s += "++++++++++++++++++++++++++++++++++\n"
    else:
        tam = 5
        for i in range(tam):
            s += f"Simulación: {i+1}\n"
            s += "PROB. ACUM. HOST 2 | PROB. ACUM. HOST 3\n"
            s += f"  {array_media[i][0]*100} % |     {array_media[i][1]*100} %\n"
            s += "++++++++++++++++++++++++++++++++++\n"
        for i in range(tam,0,-1):
            if i == tam:
                s += ".\n"
                s += ".\n"
                s += ".\n"
            s += f"Simulación: {array_media.shape[0] - i + 1}\n"
            s += "PROB. ACUM. HOST 2 | PROB. ACUM. HOST 3\n"
            s += f"    {array_media[array_media.shape[0]-i-1][0]*100} % |     {array_media[array_media.shape[0]-i-1][1]*100} %\n"
            s += "++++++++++++++++++++++++++++++++++\n"
    return s

In [13]:
n_sims = 100  # numero de computaciones de maquinas virus

n = 1000  #número de virus
p = 0.5  #probabilidad
array_sims = np.zeros((n_sims,3), dtype=np.float64)
array_sims[:,0] = n

In [14]:
array_medias = cpu_simulaciones(n,p,array_sims)

In [15]:
print(solucion_basicprobabilistic_sims(array_medias))

basicprobabilistic: 100 simulaciones
nº virus: 1000
probabilidad de pasar a host2: 50.0 %
probabilidad de pasar a host3: 50.0 %
----------------------------------------
Simulación: 1
HOST 1 | HOST 2 | HOST 3
0.0 | 504.0 | 496.0
++++++++++++++++++++++++++++++++++
Simulación: 2
HOST 1 | HOST 2 | HOST 3
0.0 | 494.0 | 506.0
++++++++++++++++++++++++++++++++++
Simulación: 3
HOST 1 | HOST 2 | HOST 3
0.0 | 468.0 | 532.0
++++++++++++++++++++++++++++++++++
Simulación: 4
HOST 1 | HOST 2 | HOST 3
0.0 | 494.0 | 506.0
++++++++++++++++++++++++++++++++++
Simulación: 5
HOST 1 | HOST 2 | HOST 3
0.0 | 523.0 | 477.0
++++++++++++++++++++++++++++++++++
.
.
.
Simulación: 96
HOST 1 | HOST 2 | HOST 3
0.0 | 479.0 | 521.0
++++++++++++++++++++++++++++++++++
Simulación: 97
HOST 1 | HOST 2 | HOST 3
0.0 | 522.0 | 478.0
++++++++++++++++++++++++++++++++++
Simulación: 98
HOST 1 | HOST 2 | HOST 3
0.0 | 525.0 | 475.0
++++++++++++++++++++++++++++++++++
Simulación: 99
HOST 1 | HOST 2 | HOST 3
0.0 | 501.0 | 499.0
++++++++++